## Hyperopt

In [1]:
'''
basically you need to define a function that returns a numeric value, 
the parameter needs to be a list
'''

'\nbasically you need to define a function that returns a numeric value, \nthe parameter needs to be a list\n'

In [3]:
import hyperopt
def objective(params):
    x = params[0]
    y = params[1]
    return -1 * (-(x + y + 12) ** 3 - y ** 4 + x ** 3)

quniform is suitable for discrete values while uniform is more suitable for continuous values

In [5]:
trials = hyperopt.Trials()
best = hyperopt.fmin(fn = objective,
                    space = [hyperopt.hp.uniform('x',-10,10), hyperopt.hp.quniform('y',-5,5,1)],
                    algo = hyperopt.tpe.suggest,
                    max_evals = 300,
                     trials = trials
                    )

100%|█████████████████████████████████████████████| 300/300 [00:02<00:00, 146.81trial/s, best loss: 263.29605578355677]


In [6]:
print(best)

{'x': -4.541300965622645, 'y': -3.0}


### demo: this is an example of using common random number and control variate

In [7]:
import hyperopt
import numpy as np
import simpy
from pynverse import inversefunc

from scipy.stats import expon, uniform, poisson
n = 100

from pynverse import inversefunc
def poisson_process(lmbda,T):
    N = poisson.rvs(lmbda * T)
    S = uniform.rvs(size = N) * T
    return np.sort(S)

def Lmbda(t):
    return 250 / np.pi * (1 - np.cos(np.pi * t / 10))

def inv(t):
    inv_1 = inversefunc(Lmbda, domain = [0,10], open_domain = [False, False])
    return inv_1(t)

def inv_method(Lmbda,t):
    Lambdat = Lmbda(t)
    S = poisson_process(1,Lambdat)
    return np.sort(inv(S))

arrivals = inv_method(Lmbda,10)
arrivals = np.concatenate((np.array([0]), arrivals))
arrival_interval = np.diff(arrivals)

lmbda = [20.,15.,1/15.] #checkout, kitchen, cost
def arrival(env,arrival_interval,checkouts,cooks,arrivals,departures,revenues,orders,checkout_time,kitchen_time):
    for i in range(len(arrival_interval)):
        dt = arrival_interval[i]
        yield env.timeout(dt)

        #print(f"customer {i} arrives at store at {env.now}")
        if len(checkouts.queue) > 5 or (len(cooks.queue) + cooks.count) > 10:
            revenues.append(-30)
            #print(f'customer {i} leaves becuase it is sick of this line')
        else:
            dt = expon.rvs(scale = np.power(lmbda,-1))
            env.process(checkout(dt,i,env,arrival_interval,checkouts,cooks,arrivals,departures,revenues,orders,checkout_time,kitchen_time))

def checkout(dt,i,env,arrival_interval,checkouts,cooks,arrivals,departures,revenues,orders,checkout_time,kitchen_time):
    price = dt[2]
    orders.append(dt[2])
    revenues.append(price)
    rqt = checkouts.request()
    # add to checkouts.queue or directly moved to users
    yield rqt # start being processed
    yield env.timeout(dt[0])
    checkout_time.append(dt[0])
    #print(f"customer {i} has finished checking out paying {price}, food order gone to kitchen ")
    checkouts.release(rqt)
    env.process(cook(dt,i,env,arrival_interval,checkouts,cooks,arrivals,departures,revenues,orders,checkout_time,kitchen_time))

def cook(dt,i,env,arrival_interval,checkouts,cooks,arrivals,departures,revenues,orders,checkout_time,kitchen_time):
    rqt = cooks.request()
    yield rqt
    yield env.timeout(dt[1])
    kitchen_time.append(dt[1])
    #print(f"customer {i} has gotten his food at {env.now} ")
    departures.append(env.now)
    cooks.release(rqt)

def system(n1,n2):
    env = simpy.Environment()
    checkouts = simpy.Resource(env,capacity = n1)
    cooks = simpy.Resource(env,capacity = n2)
    arrivals = inv_method(Lmbda,10)
    arrival_interval = np.diff(np.concatenate((np.array([0]), arrivals)))
    departures = []
    revenues = []
    orders = []
    checkout_time = []
    kitchen_time = []
    revenues.append(-n1 * 150)
    revenues.append(-n2 * 100)
    env.process(arrival(env,arrival_interval,checkouts,cooks,arrivals,departures,revenues,orders,checkout_time,kitchen_time))
    env.run()
    return sum(revenues),len(arrivals),np.mean(checkout_time),np.mean(kitchen_time),np.mean(orders)

def eval_revenue(n):
    np.random.seed(42)
    n1 = n[0]
    n2 = n[1]
    samples = [system(n1,n2) for i in range(100)]
    result = np.array(samples)
    x = result[:,0]
    z1 = result[:,1]
    z2 = result[:,2]
    z3 =  result[:,3]
    z4 = result[:,4]
    c1 = -np.cov(x,z1,ddof=1)[0,1]/np.var(z1,ddof = 1)
    c2 = -np.cov(x,z2,ddof=1)[0,1]/np.var(z2,ddof = 1)
    c3 = -np.cov(x,z3,ddof=1)[0,1]/np.var(z3,ddof = 1)
    c4 = -np.cov(x,z4,ddof=1)[0,1]/np.var(z4,ddof = 1)
    y = x + c1 * (z1 - Lmbda(10)) + c2 * (z2 - 1/20) + c3 * (z3 - 1/15) + c4 * (z4 - 15)
    return -np.mean(y)

best = hyperopt.fmin(fn = eval_revenue,
    space = [hyperopt.hp.quniform('n1',1,7,1), hyperopt.hp.quniform('n2',1,10,1)],
    algo = hyperopt.tpe.suggest,
    max_evals = 25)
print(best)

100%|███████████████████████████████████████████████| 25/25 [01:30<00:00,  3.63s/trial, best loss: -1738.7294914396998]
{'n1': 2.0, 'n2': 3.0}


## Antithetic Method

In [8]:
'''
condition:
expectation is the same, variance is the same, and also negatively correlated

algo: suppose we have 2 inputs
generate r1, r2 random state
copy r1 , r2 to get r1c, r2c
use r1, r2 to generate xorigianl numbers
use r1c, r2c to generate anti numpyers
compute : y = (xoriginal+xanti)/2

'''

'\ncondition:\nexpectation is the same, variance is the same, and also negatively correlated\n\nalgo: suppose we have 2 inputs\ngenerate r1, r2 random state\ncopy r1 , r2 to get r1c, r2c\nuse r1, r2 to generate xorigianl numbers\nuse r1c, r2c to generate anti numpyers\ncompute : y = (xoriginal+xanti)/2\n\n'

In [14]:
import numpy as np
import simpy

from scipy.stats import expon

lambda_1 = 1.
lambda_2 = 1.

def arrival(env,server,T,anti):
    scale = np.power([lambda_1, lambda_2], -1)
    for i in range(0,100):
        dt = expon.rvs(scale = scale)
        if anti == True:
            dt = expon.ppf(1-expon.cdf(dt,scale = scale), scale = scale)
        yield env.timeout(dt[0])
        env.process(service(env,server,dt,T))

def service(env,server, dt,T):
    t0  = env.now
    rqt = server.request()
    yield rqt
    yield env.timeout(dt[1])
    server.release(rqt)
    t1 = env.now
    T.append(t1-t0)
        


def simulate(anti):
    T = []
    env  = simpy.Environment()
    server = simpy.Resource(env,capacity = 1)
    env.process(arrival(env,server,T,anti))
    env.run()
    return np.mean(T)


def eval_mean(n,seed):
    np.random.seed(seed)
    T_1 = [simulate(False) for i in range(0,n)]
    np.random.seed(seed)
    T_2 = [simulate(True) for i in range(0,n)]
    T_1 = np.array(T_1)
    T_2 = np.array(T_2)
    T_means = (T_1 + T_2)/2
    return np.mean(T_means)

In [15]:
eval_mean(100,42)

7.879790550213511

### when using hyperopt, always use common random number. Else, use antithetic

## Common Random Number

In [16]:
'''
common random number is used when two things are negatively correlated.
For example,comparing difference

'''

'\ncommon random number is used when two things are negatively correlated.\nFor example,comparing difference\n\n'

In [25]:
lmbda = [20. , 10. , 30. , 10.]

def arrival(env, servers, T):
    i = 0
    while True:
        dt = expon.rvs(scale= np.power(lmbda,-1))
        if env.now + dt[0] > 1:
            break
        yield env.timeout(dt[0])
        env.process(service(env,servers,T,dt,i))
        i += 1
        
def service(env,servers,T,dt,i):
    t0 = env.now
    for j, server in enumerate(servers):
        rqt = server.request()
        yield rqt
        yield env.timeout(dt[j+1])
        server.release(rqt)
    t1 = env.now
    T.append(t1 - t0)






def simulate(n1,n2):
    T = []
    env = simpy.Environment()
    servers = [simpy.Resource(env,capacity = n1),
              simpy.Resource(env,capacity = n2),
              simpy.Resource(env,capacity = 20-n1-n2)]
    env.process(arrival(env,servers,T))
    env.run()
    return np.mean(T)

def eval_mean(n):
    np.random.seed(42)
    n1 = n[0]
    n2 = n[1]
    X = [simulate(n1,n2) for i in range(0,100)]
    return np.mean(X)

In [26]:
eval_mean([2,2])

0.3360132492314921

In [27]:
best = hyperopt.fmin(fn = eval_mean, space = [hyperopt.hp.quniform('n1',1,9,1),hyperopt.hp.quniform('n2',1,9,1)],
                    algo = hyperopt.tpe.suggest,
                    max_evals = 25)
print(best)

100%|███████████████████████████████████████████████| 25/25 [00:04<00:00,  5.94trial/s, best loss: 0.22936534918010282]
{'n1': 7.0, 'n2': 5.0}


## Control Variate

In [28]:
'''
as long as it is correlated, we can use control varaites

Algo:
Step 1: Generate a set of x samples and also a set of z samples
Step 2: c = -Cov(x,z)/Var(z)
Step 3: generate y using x + c(z - E(z))
Step 4: construc the confidence interval using the y samples

'''

'\nas long as it is correlated, we can use control varaites\n\nAlgo:\nStep 1: Generate a set of x samples and also a set of z samples\nStep 2: c = -Cov(x,z)/Var(z)\nStep 3: generate y using x + c(z - E(z))\nStep 4: construc the confidence interval using the y samples\n\n'

In [31]:
lmbda_1 = 1.
lmbda_2 = 1.

def arrival(env,server,T,IT,ST):
    scale = np.power([lmbda_1,lmbda_2],-1)
    for i in range(0,100):
        dt = expon.rvs(scale = scale)
        IT.append(dt[0])
        ST.append(dt[1])
        yield env.timeout(dt[0])
        env.process(service(env,server,dt,T))
        


def service(env,server,dt,T):
    t0 = env.now
    rqt = server.request()
    yield rqt
    yield env.timeout(dt[1])
    server.release(rqt)
    t1 = env.now
    T.append(t1 - t0)


def simulate():
    T = []
    IT = []
    ST = []
    env = simpy.Environment()
    server = simpy.Resource(env,capacity = 1)
    env.process(arrival(env,server,T, IT,ST))
    env.run()
    return np.mean(T), np.mean(IT),np.mean(ST)


def eval_mean(n):
    T_means = np.zeros(n)
    IT_means = np.zeros(n)
    ST_means = np.zeros(n)
    for i in range(0,n):
        t,it,st = simulate()
        T_means[i] = t
        IT_means[i] = it
        ST_means[i] = st
    c1 = -np.cov(T_means, IT_means)[0][1]/np.var(IT_means)
    c2 = -np.cov(T_means, IT_means)[0][1]/np.var(ST_means)
    T_tilde = T_means + c1 * (IT_means - 1/lmbda_1) + c2 * (ST_means - 1/ lmbda_2)
    return np.mean(T_tilde)

In [34]:
eval_mean(100)

6.875938457735006

## Conditioning

In [47]:
lmbda = 10
mu = 10
c = 300


def sample_M(n):
    M = np.zeros(n)
    for i in range(0,n):
        S_tmp = 0
        j = 0
        while S_tmp <= c:
            S_tmp += poisson.rvs(mu)
            j = j+1
        M[i] = j
    return M

def E_F_M(M):
    return 1 - poisson.cdf(M-1, lmbda)

M = sample_M(n)
efm = E_F_M(M)

print(np.mean(efm))
print(np.std(efm)/np.sqrt(n))

3.705963630440401e-07
7.63993591897302e-08


In [49]:
lmbda_1 = 4.
lmbda_2 = 5.

np.random.seed(42)
def arrival(env,server,N):
    for i in range(0,10):
        dt = expon.rvs(scale = np.power([lmbda_1,lmbda_2],-1))
        yield env.timeout(dt[0])
        N.append(len(server.queue) + server.count)
        env.process(service(env,server,dt))
        
def service(env,server,dt):
    rqt = server.request()
    yield rqt
    yield env.timeout(dt[1])
    server.release(rqt)
    
def simulate():
    N = []
    env = simpy.Environment()
    server = simpy.Resource(env,capacity = 1)
    env.process(arrival(env,server,N))
    env.run()
    N = np.array(N)
    E_W = (N + 1) / lmbda_2
    return np.mean(E_W)

n = 100
T = [simulate() for i in range(n)]
print(np.mean(T))
print(np.std(T, ddof = 1)/np.sqrt(n))

0.4482
0.017700299603696148


In [1]:
import hyperopt
import numpy as np
import simpy

from scipy.stats import uniform, expon
np.random.seed(42)


In [14]:
def arrivals(env,store,checkout,beefchef,veggiechef,profit, anti):
    i  = 0 
    while True:
        dt  = expon.rvs(scale = 1/40)
        U = uniform.rvs() #veggie or regular
        wait = expon.rvs(scale = 1/15) #checkout time
        if anti == True:
            dt = expon.ppf(1-expon.cdf(dt,scale = 1/40), scale = 1/40)
            U = 1- U
            wait = expon.ppf(1-expon.cdf(wait,scale = 1/15), scale = 1/15)
        yield env.timeout(dt)
        if U <= 0.2:
            order = 'veggie'
        else:
            order = 'beef'
        print(f"customer {i} has arrived at {env.now} ordering {order}")
        if env.now + dt < 8:
            if len(store.queue) + store.count < 20:
                env.process(service(env,store,checkout,beefchef,veggiechef,profit,order,wait,i))
            else:
                print(f"customer {i} left because too many people")
                profit.append(-5)
            i += 1
        else:
            break
        
            
def service(env,store,checkout,beefchef,veggiechef,profit,order,wait,i):
    rqt1 = store.request()
    rqt = checkout.request()
    yield rqt
    yield env.timeout(wait)
    print(f"customer {i} finished ordering")
    checkout.release(rqt)
    if order == 'veggie':
        env.process(kitchen_veggie(env,veggiechef,profit,i))
    else:
        env.process(kitchen_burger(env,beefchef,profit,i))
    store.release(rqt1)



def kitchen_burger(env,beefchef,profit,i):
    rqt = beefchef.request()
    yield rqt
    yield env.timeout(0.05)
    beefchef.release(rqt)
    profit.append(10)
    print(f"customer {i} got its beef burger at {env.now}")

def kitchen_veggie(env,veggiechef,profit,i):
    rqt = veggiechef.request()
    yield rqt
    yield env.timeout(0.1)
    veggiechef.release(rqt)
    profit.append(12)
    print(f"customer {i} got its veggie burger at {env.now}")

def simulate(n1,n2,n3,anti):
    env = simpy.Environment()
    checkout = simpy.Resource(env,capacity = n1)
    beefchef = simpy.Resource(env,capacity = n2)
    veggiechef = simpy.Resource(env,capacity = n3)
    store = simpy.Resource(env,capacity = 20)
    profit = []
    profit.append(-n1 * 120 - n2 * 120 - n3 * 120)
    env.process(arrivals(env,store,checkout,beefchef,veggiechef,profit, anti))
    env.run()
    return np.sum(profit)
    
    
def eval_mean(n):
    n[0] = n1
    n[1] = n2
    n[2] = n3
    np.random.seed(42)
    samples1 = [simulate(n1,n2,n3,False) for _ in range(50)]
    np.random.seed(42)
    samples2 = [simulate(n1,n2,n3,True) for _ in range(50)]
    samples = (samples1 + samples2)/2
    return np.mean(samples)
    

In [16]:
simulate(4,3,2,False)

customer 0 has arrived at 0.05579657552054569 ordering beef
customer 1 has arrived at 0.06330473182422766 ordering beef
customer 2 has arrived at 0.07110627456960948 ordering beef
customer 0 finished ordering
customer 3 has arrived at 0.0911829587324806 ordering beef
customer 2 finished ordering
customer 3 finished ordering
customer 1 finished ordering
customer 0 got its beef burger at 0.13368142919848117
customer 2 got its beef burger at 0.15621534385493024
customer 3 got its beef burger at 0.16443629041663615
customer 4 has arrived at 0.1653146392612857 ordering beef
customer 4 finished ordering
customer 1 got its beef burger at 0.18368142919848118
customer 5 has arrived at 0.2160272624542688 ordering beef
customer 4 got its beef burger at 0.2253782724694871
customer 6 has arrived at 0.2562451238025163 ordering beef
customer 6 finished ordering
customer 7 has arrived at 0.26406763749692846 ordering beef
customer 8 has arrived at 0.27152400449515196 ordering veggie
customer 6 got its 

1750